# Advanced Applications of Mutate

## Map and apply

In [1]:
import pandas as pd
from dfply import *
import matplotlib.pylab as plt
%matplotlib inline

## Hiding stack traceback

We hide the exception traceback for didactic reasons (code source: [see this post](https://stackoverflow.com/questions/46222753/how-do-i-suppress-tracebacks-in-jupyter)).  Don't run this cell if you want to see a full traceback.

In [2]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Data set

We will be using two of the data sets provided by the Museam of Modern Art (MoMA) in this lecture.  Make sure that you have downloaded each repository.  [Download Instructions](./get_MOMA_data.ipynb)

#### MoMA Artists

In [3]:
artists = pd.read_csv("./data/Artists.csv")
artists.head(2)

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,1,Robert Arneson,"American, 1930–1992",American,Male,1930,1992,NaN,NaN
1,2,Doroteo Arnaiz,"Spanish, born 1936",Spanish,Male,1936,0,NaN,NaN


# Transforming columns with the `map` and `apply` methods

Next, we will take a look at two useful `pandas Series` methods that allow us to apply very general transformations: `map` and `apply`.

## Transforming a column with `map`

`df.col.map` can be used to

* Apply a translation `dict`
* Apply a function
* Apply a `pd.Series`

In [4]:
artists.Gender.value_counts()

Male          9762
Female        2300
male            15
Non-Binary       2
female           1
Non-binary       1
Name: Gender, dtype: int64

#### `map`ping a translation `dict`

In [6]:
new_gender = {'Male':'m', 'Female':'f', 'male':'m', 'female':'f', 'Non-Binary':'nb', 'Non-binary':'nb'}
(artists
 >> select(X.Gender)
 >> mutate(new_gender = X.Gender.map(new_gender))
 >> sample(5)
)

,Gender,new_gender
8035,NaN,NaN
9784,Male,m
6768,Male,m
6219,Female,f
14628,Female,f


#### Setting a default with `collections.defaultdict`

In [7]:
from collections import defaultdict

from_america = defaultdict(lambda: 'Not America')
from_america.update({'American':'America'})

#### Applying the `defaultdict`

In [8]:
(artists
 >> select(X.Nationality)
 >> mutate(from_america = X.Nationality.map(from_america))
 >> head(3)
)

,Nationality,from_america
0,American,America
1,Spanish,Not America
2,American,America


#### `map`ping a simple function

In [9]:
(artists
 >> select(X.Nationality)
 >> mutate(from_USA = X.Nationality.map(lambda n: 'USA' if n == 'American' else 'Other'))
 >> head(3)
)

,Nationality,from_USA
0,American,USA
1,Spanish,Other
2,American,USA


## Be sure to `apply` yourself!

* `df.col.apply` is used to apply any function to a column.
    * Including positional and keyword arguments
* Could literally be used to perform *any* mutation

#### Applying a unary function

In [10]:
century = lambda year_string: (int(year_string)//100)*100

(artists
 >> select(X.BeginDate)
 >> mutate(century_of_birth = X.BeginDate.apply(century))
 >> head(3)
)

,BeginDate,century_of_birth
0,1930,1900
1,1936,1900
2,1941,1900


## Using anonymous functions

* There is no need to name a `lambda`
* An embedded `lambda` is called an **anonymous function**

In [11]:
(artists
 >> select(X.EndDate)
 >> mutate(new_end_date = (X.EndDate
                           .apply(lambda y: y if int(y) > 0 else np.nan)
                           .astype('Int64')))
 >> head(2)
)

,EndDate,new_end_date
0,1992,1992
1,0,<NA>


## `apply` or `map`

* Use `map` for simple functions
* Use `apply` when adding additional arguments

#### MoMA Artwork

In [12]:
from more_dfply import fix_names

artwork = (pd.read_csv("./data/Artworks.csv")
           >> fix_names
           >> mutate(id = X.index + 1)
          )
artwork.head(2)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Circumference_cm,Depth_cm,Diameter_cm,Height_cm,Length_cm,Weight_kg,Width_cm,Seat_Height_cm,Duration_sec,id
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN,1
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN,2


#### Setting a positional argument

We want to apply `round(val, 1)`


In [13]:
(artwork
 >> select(X.Height_cm)
 >> mutate(rounded_height = X.Height_cm.apply(round, args=(1,)))
 >> head(3)
)

,Height_cm,rounded_height
0,48.6000,48.6
1,40.6401,40.6
2,34.3000,34.3


#### Setting a keyword argument

We want to apply `logp1(val, base=n)`

In [14]:
from math import log, e

log1p = lambda num, base=e: log(num + 1, base)
(artwork
 >> select(X.Height_cm)
 >> mutate(log10_plus_1 = X.Height_cm.apply(log1p, base = 10),
           log2_plus_1 = X.Height_cm.apply(log1p, base = 2),
           ln_plus_1 = X.Height_cm.apply(log1p, base = e))
 >> head(3)
)

,Height_cm,log10_plus_1,log2_plus_1,ln_plus_1
0,48.6000,1.695482,5.632268,3.903991
1,40.6401,1.619512,5.379902,3.729064
2,34.3000,1.547775,5.141596,3.563883


## <font color="red"> Exercise 3.3.1 </font>

An **Indicator column** for a category contains 1 for the rows that match that label and 0 otherwise.  The `exhibitions` dataframe.  Complete the following tasks.

1. Use `exhibitions.ExhibitionRole.unique()` to get a list of unique columns.
2. Use `mutate` and `map` with `defaultdict` to create an indicator column for each category (ignore missing rows).
3. Comment on the quality of your solution, especially in light of the [DRY principle](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)

#### MoMA Exhibitions

In [16]:
dat_cols = ['ExhibitionBeginDate', 'ExhibitionEndDate', 'ConstituentBeginDate' ,'ConstituentEndDate']
exhibitions = pd.read_csv('./data/MoMAExhibitions1929to1989.csv', 
                          encoding="ISO-8859-1",
                          parse_dates=dat_cols)
exhibitions.head(2)

,ExhibitionID,ExhibitionNumber,ExhibitionTitle,ExhibitionCitationDate,ExhibitionBeginDate,ExhibitionEndDate,ExhibitionSortOrder,ExhibitionURL,ExhibitionRole,ExhibitionRoleinPressRelease,...,Institution,Nationality,ConstituentBeginDate,ConstituentEndDate,ArtistBio,Gender,VIAFID,WikidataID,ULANID,ConstituentURL
0,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Curator,Director,...,NaN,American,1902,1981,"American, 19021981",Male,109252853.0,Q711362,500241556.0,moma.org/artists/9168
1,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Artist,Artist,...,NaN,French,1839,1906,"French, 18391906",Male,39374836.0,Q35548,500004793.0,moma.org/artists/1053


In [18]:
exhibitions.ExhibitionRole.unique()

array(['Curator', 'Artist', nan, 'Arranger', 'Installer',
       'Competition Judge', 'Designer', 'Preparer'], dtype=object)

In [29]:
is_Curator = defaultdict(lambda: 0)
is_Curator.update({'Curator': 1 })

is_Artist = defaultdict(lambda: 0)
is_Artist.update({'Artist': 1})

is_Arranger = defaultdict(lambda: 0)
is_Arranger.update({'Arranger': 1 })

is_Installer = defaultdict(lambda: 0)
is_Installer.update({'Installer': 1})

is_Competition_Judge = defaultdict(lambda: 0)
is_Competition_Judge.update({'Competition Judge': 1 })

is_Designer = defaultdict(lambda: 0)
is_Designer.update({'Designer': 1})

is_Preparer = defaultdict(lambda: 0)
is_Preparer.update({'Preparer': 1 })

In [41]:
(exhibitions
 >> mutate(is_Curator = X.ExhibitionRole.map(is_Curator),
           is_Artist = X.ExhibitionRole.map(is_Artist),
           is_Arranger = X.ExhibitionRole.map(is_Arranger),
           is_Installer = X.ExhibitionRole.map(is_Installer),
           is_Competition_Judge = X.ExhibitionRole.map(is_Competition_Judge),
           is_Designer = X.ExhibitionRole.map(is_Designer),
           is_Preparer = X.ExhibitionRole.map(is_Preparer))
 >> sample(7)
)

,ExhibitionID,ExhibitionNumber,ExhibitionTitle,ExhibitionCitationDate,ExhibitionBeginDate,ExhibitionEndDate,ExhibitionSortOrder,ExhibitionURL,ExhibitionRole,ExhibitionRoleinPressRelease,...,WikidataID,ULANID,ConstituentURL,is_Curator,is_Artist,is_Arranger,is_Installer,is_Competition_Judge,is_Designer,is_Preparer
15958,3977.0,624,70 Photographers Look at New York,"[MoMA Exh. #624, November 27, 1957-April 15, 1...",1957-11-27,1958-04-15,702.0,moma.org/calendar/exhibitions/2439,Artist,Artist,...,Q776395,NaN,moma.org/artists/47540,0,1,0,0,0,0,0
16534,79.0,637,20th Century Design from the Museum Collection,"[MoMA Exh. #637, December 17, 1958-February 23...",1958-12-17,1959-02-23,716.0,moma.org/calendar/exhibitions/2729,Artist,Artist,...,NaN,NaN,moma.org/artists/9758,0,1,0,0,0,0,0
24625,4135.0,1021a,25 Recent Acquisitions,"[MoMA Exh. #1021a, January 17-March 18, 1973]",1973-01-22,1973-03-18,1151.0,moma.org/calendar/exhibitions/2547,Artist,Artist,...,Q2000212,500349810.0,moma.org/artists/66479,0,1,0,0,0,0,0
24370,4343.0,1007b,Prints for Collectors,"[MoMA Exh. #1007b, June 26-September 15, 1972]",1972-06-26,1972-09-15,1133.0,moma.org/calendar/exhibitions/3520,Curator,Organizer,...,NaN,NaN,moma.org/artists/8847,1,0,0,0,0,0,0
26751,10545.0,1158a,Rooms,"[MoMA Exh. #1158a, December 7, 1976-February 6...",1976-12-07,1977-02-06,1308.0,moma.org/calendar/exhibitions/3580,Artist,Artist,...,Q478264,500123705.0,moma.org/artists/207,0,1,0,0,0,0,0
21570,4290.0,845,Steichen Gallery Reinstallation,"[MoMA Exh. #845, October 25, 1967-unknown clos...",1967-10-25,NaT,948.0,moma.org/calendar/exhibitions/2609,Artist,Artist,...,Q156567,500115484.0,moma.org/artists/1000,0,1,0,0,0,0,0
1374,3018.0,39,African Negro Art,"[MoMA Exh. #39, March 18-May 19, 1935]",1935-03-18,1935-05-19,64.0,moma.org/calendar/exhibitions/2937,Curator,Selector,...,Q6136905,500258108.0,moma.org/artists/14845,1,0,0,0,0,0,0


> This is definetley not following the DRY principle. There is a lot of repetition of essentially the same code.